In [62]:
import Parser as xml_parser
import numpy as np
import pandas as pd
import turbodbc

In [63]:
def create_connection_string_turbo(server, database):
    options = turbodbc.make_options(prefer_unicode=True)
    constr = 'Driver={ODBC Driver 13 for SQL Server};Server=' + \
        server + ';Database=' + database + ';Trusted_Connection=yes;'
    con = turbodbc.connect(connection_string=constr, turbodbc_options=options)
    return con

In [64]:
tree = xml_parser.get_tree("WuerthSchweiz-4327-2018-01-de.xml")

In [65]:
tree

In [66]:
Parser = xml_parser.XML_Parser(tree)

In [67]:
Parser.parse_xml()

In [68]:
Parser.dict_to_df("id")

In [69]:
df = Parser.DF[['ArtikelId','Art_Nr_Hersteller_Firma', 'Art_Nr_EAN',
           'Art_Txt_Kurz', 'Art_Txt_Lang', 'Preis']].copy()

In [70]:
df['Preis'] = df['Preis'].astype(np.float)
df['Art_Nr_EAN'] = df['Art_Nr_EAN'].astype("str")

In [71]:
df.head()

,,,ArtikelId,Art_Nr_Hersteller_Firma,Art_Nr_EAN,Art_Txt_Kurz,Art_Txt_Lang,Preis
016017_105005__200,,,016017_105005__200,NaN,4011231219532,Stahl gelb verzinkt kopflochgebohrt Senkkopf Z,SHR-DBL-SEKPF-KB-Z4-(A2C)-7X105- Länge: 105.0...,55.90
016017_120005__200,,,016017_120005__200,NaN,4011231219556,Stahl gelb verzinkt kopflochgebohrt Senkkopf Z,SHR-DBL-SEKPF-KB-Z4-(A2C)-7X120- Länge: 120.0...,68.25
016017_140005__200,,,016017_140005__200,NaN,4011231219570,Stahl gelb verzinkt kopflochgebohrt Senkkopf Z,SHR-DBL-SEKPF-KB-Z4-(A2C)-7X140- Länge: 140.0...,75.80
016017_165005__200,,,016017_165005__200,NaN,4011231219594,Stahl gelb verzinkt kopflochgebohrt Senkkopf Z,SHR-DBL-SEKPF-KB-Z4-(A2C)-7X165- Länge: 165.0...,87.35
016017_85_005__200,,,016017_85_005__200,NaN,4011231219617,Stahl gelb verzinkt kopflochgebohrt Senkkopf Z,SHR-DBL-SEKPF-KB-Z4-(A2C)-7X85- Länge: 85.000 mm,50.50


In [72]:
query = """
set nocount on;

SELECT  [Artikelnummer]
      ,[Artikel Beschreibung]
      ,[Lieferantenname]
      ,[Bruttoverkauf]
      ,[Standardkosten]
      ,[Lieferantennummer]
      ,EAN = Barcode
into #temp
FROM [AnalystCM].[dbo].[LOOKUP_ArtikelstammCRHT]
where [Einmalartikel (J/N)] = 0 and [Artikel Status] = '02'

select idItemOrigin
,AvgPrice = Avg(Sales / case when Quantity = 0 then NULL else Quantity end)
,WAvgPrice = sum(Sales) / case when sum(Quantity) = 0 then NULL else sum(Quantity) end
into #avgprice
from crhbusadwh01.infopool.fact.sales
where date > dateadd(month,-12, getdate())
group by idItemOrigin

select [Artikelnummer]
      ,[Artikel Beschreibung]
      ,[Lieferantenname]
      ,[Bruttoverkauf]
      ,[Standardkosten]
      ,[ArtikelnummerLieferant] = [Lieferantennummer]
      ,EAN
      ,Durchschnittspreis = AvgPrice
      ,GDurchschnittspreis = WAvgPrice
from #temp t
    left join #avgprice a on t.Artikelnummer = a.idItemOrigin collate Latin1_General_CI_AS

drop table #temp
drop table #avgprice
"""

In [73]:
con = create_connection_string_turbo("CRHBUSADWH51", "Operations")
df_server = pd.read_sql(query, con)
df_server['Bruttoverkauf'] = df_server['Bruttoverkauf'].astype("float")
df_server['Standardkosten'] = df_server['Standardkosten'].astype("float")
df_server['EAN'] = df_server['EAN'].astype("str")

In [74]:
df = df.merge(df_server, how="left", left_on="Art_Nr_EAN", right_on="EAN")

In [75]:
df.to_excel("Würth.xlsx", index=False)